In [1]:
import cv2
import numpy as np
import tensorflow as tf
from keras.datasets import mnist
from keras.models import Sequential
from keras.layers import Dense, Flatten
from keras.utils import to_categorical
from keras.layers import Conv2D, MaxPooling2D
from keras.layers import Dropout
from PIL import Image
from keras.preprocessing import image

In [5]:
def train_model_base(model, x_train, y_train, x_test, y_test, epochs=100, batch_size=200):
    y_train = to_categorical(y_train)
    y_test = to_categorical(y_test)
    model.fit(x_train, y_train,
              batch_size=batch_size,
              epochs=epochs,
              validation_split=0.2, # Доля валидационной выборки для контроля переобучения
              verbose=2) #verbose=0 ничего не покажет, verbose=1 покажет анимированный индикатор выполнения (progress_bar), verbose=2 будет просто указать номер эпохи

In [6]:
def predict_dense_base(model, img):
    print("shape:", img.shape)  # Добавим вывод формы входных данных
    # Получаем вероятности для каждого класса
    probabilities = model.predict(img)
    # Используем argmax для получения индекса класса с наибольшей вероятностью
    return np.argmax(probabilities)

In [7]:
def print_metrics_base(model):
    history_dict = model.history.history
    best_epoch = np.argmax(history_dict['val_accuracy']) + 1

    print("Метрики обучения:")
    print("---------------------------------------------------------")
    print(f"Максимальная точность на тренировочном наборе: {max(history_dict['accuracy']):.4f}")
    print(f"Минимальная потеря на тренировочном наборе: {min(history_dict['loss']):.4f}")
    print(f"Максимальная точность на валидационном наборе: {max(history_dict['val_accuracy']):.4f}")
    print(f"Минимальная потеря на валидационном наборе: {min(history_dict['val_loss']):.4f}")
    print(f"Наивысшая точность достигнута на эпохе: {best_epoch}")

In [57]:
# DenseModel
def build_dense_model(input_shape=(28, 28), num_classes=10):
    # Создаем последовательную модель нейронной сети
    model = Sequential()
    # Входной полносвязный слой
    # - 800 нейронов: определяет количество нейронов в слое
    # - input_dim=784: указывает на количество входов в каждый нейрон, в данном случае, 784, так как входные данные представляют собой изображения размером 28x28 пикселей, которые были преобразованы в одномерный массив размером 784 пикселя
    # - activation="relu": функция активации ReLU (Rectified Linear Unit) применяется для добавления нелинейности в сеть
    model.add(Dense(800, input_dim=784, activation="relu"))
    # Выходной полносвязный слой
    # - 10 нейронов: количество нейронов в выходном слое соответствует количеству классов, в данном случае, 10 классов (цифры от 0 до 9)
    # - activation="softmax": функция активации Softmax применяется для преобразования выходов сети в вероятности принадлежности к каждому классу
    model.add(Dense(10, activation="softmax"))
    # Компиляция модели
    # - loss="categorical_crossentropy": функция потерь, используемая для оценки ошибки сети; в данном случае, используется категориальная перекрестная энтропия, так как у нас задача классификации с несколькими классами
    # - optimizer="SGD": оптимизатор, используемый для обучения сети; в данном случае, используется стохастический градиентный спуск (SGD)
    # - metrics=["accuracy"]: метрики, используемые для оценки производительности сети во время обучения; в данном случае, используется метрика точности (accuracy)
    model.compile(loss="categorical_crossentropy", optimizer="SGD", metrics=["accuracy"])

    return model

def train_dense_model(model, x_train, y_train, x_test, y_test, epochs=100, batch_size=64):
    train_model_base(model, x_train, y_train, x_test, y_test, epochs, batch_size)  # Вызываем метод из класса otherModel

def print_dense_metrics(model):
    print_metrics_base(model)   # Вызываем метод из класса otherModel

def transformation_img_dense(img_path):
    img = image.load_img(img_path, target_size=(28, 28), color_mode="grayscale")
    # Преобразуем картинку в массив
    x = image.img_to_array(img)
    x = x.reshape(1,784)
    # Нормализуем изображение
    x = 255 - x
    x /= 255
    # Выпрямляем изображение в одномерный массив
    x = x.flatten()
    # Добавляем размерность батча (1, 784)
    x = np.expand_dims(x, axis=0)
    return x

def predict_dense(model, img):
    print("Предсказанное число ", predict_dense_base(model, transformation_img_dense(img)))



In [56]:
# ConvModel:

def build_conv_model(input_shape=(28, 28, 1), num_classes=10):
    # Создаем последовательную модель нейронной сети
    model = Sequential()
    # Первый сверточный слой
    # 28 - количество фильтров, (3, 3) - размер ядра свертки, padding='same' - добавляет нулевые значения входам, чтобы размер выхода был таким же, как и у входа
    # input_shape=(28, 28, 1) - размерность входных данных (ширина, высота, количество каналов)
    # activation='relu' - функция активации ReLU (Rectified Linear Unit)
    model.add(Conv2D(28, (3, 3), padding='same', input_shape=(28, 28, 1), activation='relu'))
    # Второй сверточный слой
    # Аналогично первому слою, но без указания input_shape, так как это уже продолжение предыдущего слоя
    model.add(Conv2D(28, (3, 3), activation='relu', padding='same'))
    # Первый слой подвыборки
    # pool_size=(2, 2) - размер окна для уменьшения размерности
    model.add(MaxPooling2D(pool_size=(2, 2)))
    # Слой регуляризации Dropout
    # 0.25 - доля входных единиц, которые будут исключены случайным образом на каждом обновлении во время обучения для предотвращения переобучения
    model.add(Dropout(0.25))
    # Третий сверточный слой
    model.add(Conv2D(2 * 28, (3, 3), padding='same', activation='relu'))
    # Четвертый сверточный слой
    model.add(Conv2D(2 * 28, (3, 3), activation='relu'))
    # Второй слой подвыборки
    model.add(MaxPooling2D(pool_size=(2, 2)))
    # Слой регуляризации Dropout
    model.add(Dropout(0.25))
    # Слой преобразования данных из 2D представления в плоское
    model.add(Flatten())
    # Полносвязный слой для классификации
    # 8 * 28 - количество нейронов в слое
    model.add(Dense(8 * 28, activation='relu'))
    # Слой регуляризации Dropout
    model.add(Dropout(0.5))
    # Выходной полносвязный слой
    # 10 - количество классов для классификации
    model.add(Dense(10, activation='softmax'))
    # Компилируем модель
    # loss='categorical_crossentropy' - функция потерь для многоклассовой классификации
    # optimizer='adam' - оптимизатор Adam
    # metrics=['accuracy'] - метрика, используемая для оценки производительности модели
    model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])

    return model

def transformation_img_conv(img_path):
    img = image.load_img(img_path, target_size=(28, 28), color_mode="grayscale")
    # Преобразуем картинку в массив
    x = image.img_to_array(img)
    x= 255 - x
    # Нормализуем изображение
    x /= 255
    # для второй нейронки
    x = np.expand_dims(x, axis=0)
    return x

#def train_conv_model(model, x_train, y_train, x_test, y_test, epochs=15, batch_size=64):
#    train_model_base(model, x_train, y_train, x_test, y_test, epochs, batch_size)  # Вызываем метод из класса otherModel

def predict_conv(model, img):
    print("Предсказанное число (Dense Model):", predict_dense_base(model, transformation_img_conv(img)))

In [3]:
# Загрузка данных MNIST
(x_train, y_train), (x_test, y_test) = mnist.load_data()

# Нормализация изображений
x_train = x_train / 255.0
x_test = x_test / 255.0

In [ ]:
# Создание экземпляров классов моделей
#dense_model = DenseModel()

In [ ]:
#conv_model = ConvModel()

In [45]:
# Обучение полносвязной нейронной сети
model = build_dense_model()

# Преобразовать форму входных данных
x_train_flat = x_train.reshape(x_train.shape[0], -1)
x_test_flat = x_test.reshape(x_test.shape[0], -1)

train_dense_model(model, x_train_flat, y_train, x_test_flat, y_test)

Epoch 1/100
750/750 - 7s - loss: 0.8800 - accuracy: 0.8061 - val_loss: 0.4604 - val_accuracy: 0.8886 - 7s/epoch - 10ms/step
Epoch 2/100
750/750 - 5s - loss: 0.4290 - accuracy: 0.8885 - val_loss: 0.3563 - val_accuracy: 0.9053 - 5s/epoch - 7ms/step
Epoch 3/100
750/750 - 7s - loss: 0.3591 - accuracy: 0.9019 - val_loss: 0.3168 - val_accuracy: 0.9145 - 7s/epoch - 9ms/step
Epoch 4/100
750/750 - 6s - loss: 0.3238 - accuracy: 0.9096 - val_loss: 0.2932 - val_accuracy: 0.9191 - 6s/epoch - 8ms/step
Epoch 5/100
750/750 - 7s - loss: 0.3003 - accuracy: 0.9168 - val_loss: 0.2757 - val_accuracy: 0.9242 - 7s/epoch - 9ms/step
Epoch 6/100
750/750 - 5s - loss: 0.2818 - accuracy: 0.9217 - val_loss: 0.2615 - val_accuracy: 0.9273 - 5s/epoch - 7ms/step
Epoch 7/100
750/750 - 6s - loss: 0.2666 - accuracy: 0.9263 - val_loss: 0.2494 - val_accuracy: 0.9333 - 6s/epoch - 8ms/step
Epoch 8/100
750/750 - 6s - loss: 0.2533 - accuracy: 0.9302 - val_loss: 0.2385 - val_accuracy: 0.9342 - 6s/epoch - 8ms/step
Epoch 9/100
750

In [46]:
print_metrics_base(model)

Метрики обучения:
---------------------------------------------------------
Максимальная точность на тренировочном наборе: 0.9910
Минимальная потеря на тренировочном наборе: 0.0410
Максимальная точность на валидационном наборе: 0.9760
Минимальная потеря на валидационном наборе: 0.0836
Наивысшая точность достигнута на эпохе: 100


In [54]:
def dense(model, path):
    predict_dense(model, path)

In [58]:
import os

# Укажите путь к папке, которую вы хотите просканировать
folder_path = '/content/'

# Допустимые расширения файлов изображений
image_extensions = ['.jpg', '.jpeg', '.png']

# Цикл для перебора всех файлов в папке
for root, dirs, files in os.walk(folder_path):
    for file_name in files:
        # Полный путь к файлу
        file_path = os.path.join(root, file_name)

        # Получение расширения файла
        file_extension = os.path.splitext(file_name)[1].lower()

        # Проверка, является ли файл изображением
        if file_extension in image_extensions:
            print(f"-------{file_path}---------")
            dense(model, file_path)

-------/content/num5_1.png---------
shape: (1, 784)
1/1 [==============================] - 0s 59ms/step
Предсказанное число  5
-------/content/num0_1.png---------
shape: (1, 784)
1/1 [==============================] - 0s 26ms/step
Предсказанное число  0
-------/content/num4.png---------
shape: (1, 784)
1/1 [==============================] - 0s 26ms/step
Предсказанное число  4
-------/content/num7_3.png---------
shape: (1, 784)
1/1 [==============================] - 0s 23ms/step
Предсказанное число  6
-------/content/num1_1.png---------
shape: (1, 784)
1/1 [==============================] - 0s 24ms/step
Предсказанное число  1
-------/content/num5_3.png---------
shape: (1, 784)
1/1 [==============================] - 0s 34ms/step
Предсказанное число  5
-------/content/num8_1.png---------
shape: (1, 784)
1/1 [==============================] - 0s 34ms/step
Предсказанное число  2
-------/content/num6_1.png---------
shape: (1, 784)
1/1 [==============================] - 0s 31ms/step
Предсказа

In [27]:
# Изменение формы входных данных для сверточной сети
x_train_conv = x_train.reshape(-1, 28, 28, 1)
x_test_conv = x_test.reshape(-1, 28, 28, 1)

# Обучение сверточной нейронной сети
model_conv = build_conv_model()

In [28]:
train_model_base(model_conv, x_train_conv, y_train, x_test_conv, y_test)

Epoch 1/100
240/240 - 151s - loss: 0.3525 - accuracy: 0.8845 - val_loss: 0.0607 - val_accuracy: 0.9817 - 151s/epoch - 630ms/step
Epoch 2/100
240/240 - 144s - loss: 0.0943 - accuracy: 0.9717 - val_loss: 0.0512 - val_accuracy: 0.9856 - 144s/epoch - 601ms/step
Epoch 3/100
240/240 - 152s - loss: 0.0676 - accuracy: 0.9792 - val_loss: 0.0396 - val_accuracy: 0.9885 - 152s/epoch - 634ms/step
Epoch 4/100
240/240 - 144s - loss: 0.0569 - accuracy: 0.9830 - val_loss: 0.0315 - val_accuracy: 0.9912 - 144s/epoch - 598ms/step
Epoch 5/100
240/240 - 142s - loss: 0.0495 - accuracy: 0.9848 - val_loss: 0.0298 - val_accuracy: 0.9917 - 142s/epoch - 590ms/step
Epoch 6/100
240/240 - 140s - loss: 0.0395 - accuracy: 0.9874 - val_loss: 0.0317 - val_accuracy: 0.9918 - 140s/epoch - 583ms/step
Epoch 7/100
240/240 - 140s - loss: 0.0385 - accuracy: 0.9883 - val_loss: 0.0329 - val_accuracy: 0.9913 - 140s/epoch - 584ms/step
Epoch 8/100
240/240 - 148s - loss: 0.0326 - accuracy: 0.9895 - val_loss: 0.0280 - val_accuracy: 0

In [29]:
print_metrics_base(model_conv)

Метрики обучения:
---------------------------------------------------------
Максимальная точность на тренировочном наборе: 0.9987
Минимальная потеря на тренировочном наборе: 0.0036
Максимальная точность на валидационном наборе: 0.9956
Минимальная потеря на валидационном наборе: 0.0244
Наивысшая точность достигнута на эпохе: 88


In [60]:
def conv(model, path):
    predict_conv(model, path)

In [62]:
import pickle

# Сохранение модели в файл
filename = 'model_rupasov.pkl'
with open(filename, 'wb') as file:
    pickle.dump(model, file)

In [63]:
# Сохранение модели в файл
filename = 'model_conv_rupasov.pkl'
with open(filename, 'wb') as file:
    pickle.dump(model_conv, file)

In [41]:
# Загрузка модели из файла
filename = 'model_rupasov.pkl'
with open(filename, 'rb') as file:
    model111 = pickle.load(file)

In [42]:
# Загрузка модели из файла
filename = 'model_conv_rupasov.pkl'
with open(filename, 'rb') as file:
    model222 = pickle.load(file)

In [59]:
import os

# Укажите путь к папке, которую вы хотите просканировать
folder_path = '/content/'

# Допустимые расширения файлов изображений
image_extensions = ['.jpg', '.jpeg', '.png']

# Цикл для перебора всех файлов в папке
for root, dirs, files in os.walk(folder_path):
    for file_name in files:
        # Полный путь к файлу
        file_path = os.path.join(root, file_name)

        # Получение расширения файла
        file_extension = os.path.splitext(file_name)[1].lower()

        # Проверка, является ли файл изображением
        if file_extension in image_extensions:
            print(f"-------{file_path}---------")
            dense(model111, file_path)

-------/content/num5_1.png---------
shape: (1, 784)
1/1 [==============================] - 0s 85ms/step
Предсказанное число  5
-------/content/num0_1.png---------
shape: (1, 784)
1/1 [==============================] - 0s 32ms/step
Предсказанное число  0
-------/content/num4.png---------
shape: (1, 784)
1/1 [==============================] - 0s 34ms/step
Предсказанное число  4
-------/content/num7_3.png---------
shape: (1, 784)
1/1 [==============================] - 0s 32ms/step
Предсказанное число  6
-------/content/num1_1.png---------
shape: (1, 784)
1/1 [==============================] - 0s 35ms/step
Предсказанное число  1
-------/content/num5_3.png---------
shape: (1, 784)
1/1 [==============================] - 0s 34ms/step
Предсказанное число  5
-------/content/num8_1.png---------
shape: (1, 784)
1/1 [==============================] - 0s 41ms/step
Предсказанное число  2
-------/content/num6_1.png---------
shape: (1, 784)
1/1 [==============================] - 0s 34ms/step
Предсказа

In [61]:
import os

# Укажите путь к папке, которую вы хотите просканировать
folder_path = '/content/'

# Допустимые расширения файлов изображений
image_extensions = ['.jpg', '.jpeg', '.png']

# Цикл для перебора всех файлов в папке
for root, dirs, files in os.walk(folder_path):
    for file_name in files:
        # Полный путь к файлу
        file_path = os.path.join(root, file_name)

        # Получение расширения файла
        file_extension = os.path.splitext(file_name)[1].lower()

        # Проверка, является ли файл изображением
        if file_extension in image_extensions:
            print(f"-------{file_path}---------")
            conv(model222, file_path)

-------/content/num5_1.png---------
shape: (1, 28, 28, 1)
1/1 [==============================] - 0s 112ms/step
Предсказанное число (Dense Model): 5
-------/content/num0_1.png---------
shape: (1, 28, 28, 1)
1/1 [==============================] - 0s 26ms/step
Предсказанное число (Dense Model): 0
-------/content/num4.png---------
shape: (1, 28, 28, 1)
1/1 [==============================] - 0s 23ms/step
Предсказанное число (Dense Model): 4
-------/content/num7_3.png---------
shape: (1, 28, 28, 1)
1/1 [==============================] - 0s 22ms/step
Предсказанное число (Dense Model): 3
-------/content/num1_1.png---------
shape: (1, 28, 28, 1)
1/1 [==============================] - 0s 23ms/step
Предсказанное число (Dense Model): 1
-------/content/num5_3.png---------
shape: (1, 28, 28, 1)
1/1 [==============================] - 0s 23ms/step
Предсказанное число (Dense Model): 5
-------/content/num8_1.png---------
shape: (1, 28, 28, 1)
1/1 [==============================] - 0s 26ms/step
Предсказа